In [1]:
import numpy as np
# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

In [5]:
audio_path = "./project/genres/hiphop/hiphop.00000.au"
y, sr = librosa.load(audio_path)

In [20]:
# S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
# We have to truncate the data for valid layer dimensions
TRUNC_VAL = 1288


y_harmonic, y_percussive = librosa.effects.hpss(y)
S_harmonic = librosa.feature.melspectrogram(y_harmonic, sr=sr)[:,:TRUNC_VAL]
S_percussive = librosa.feature.melspectrogram(y_percussive, sr=sr)[:,:TRUNC_VAL]

print(f"Harmonic shape: {y_harmonic.shape}")
print(f"Spectogram Harmonic shape: {S_harmonic.shape}")
print(f"stacked shape: {np.stack([S_harmonic, S_percussive], axis=-1).shape}")

Harmonic shape: (661504,)
Spectogram Harmonic shape: (128, 1288)
stacked shape: (128, 1288, 2)


In [234]:
genres = ["hiphop","blues"]#,"classical","country","disco","jazz","metal","pop","reggae","rock"]
num_classes = len(genres)
stacks_y = []
stacks_separates = []
y_list = []
raw_y_s = []
for i, genre in enumerate(genres):
    for k in range(2):
        path = f"./project/genres/{genre}/{genre}.0000{k}.au"
        y, sr = librosa.load(audio_path)
        raw_y_s.append(y)
        stacks_y.append(librosa.feature.melspectrogram(y, sr=sr)[:,:TRUNC_VAL])
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        S_harmonic_i = librosa.feature.melspectrogram(y_harmonic, sr=sr)[:,:TRUNC_VAL]
        S_percussive_i = librosa.feature.melspectrogram(y_percussive, sr=sr)[:,:TRUNC_VAL]
        stacks_separates.append(np.stack([S_harmonic_i, S_percussive_i], axis=-1))
        y_list.append(i)
x = np.stack(stacks_y, axis=0)
x = np.expand_dims(x, axis=-1)
x_separates = np.stack(stacks_separates, axis=0)
y = np.array(y_list)
raw_y = np.stack(raw_y_s, axis=0)
raw_y.shape

(4, 661504)

In [228]:
from keras.models import Sequential
from keras.layers import Dense, Conv3D, MaxPooling2D, Conv2D, Flatten, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
from keras.optimizers import Adam
from keras.backend import gradients, eval, get_value
from keras import backend as K
print(x.shape)
print(x_separates.shape)
M, H, W, C = x.shape
random_test = np.random.randn(M, H, W, C)

(4, 128, 1288, 1)
(4, 128, 1288, 2)


In [229]:
import tensorflow as tf
sess = tf.Session()

K.set_session(sess)


model = Sequential()
model.add(Conv2D(
    filters=16,
    kernel_size=5,
    data_format="channels_last",
    input_shape=(128,TRUNC_VAL,1)
))
model.add(BatchNormalization())
model.add(MaxPooling2D(padding="valid"))
model.add(Conv2D(
    filters=32,
    kernel_size=5
))
model.add(BatchNormalization())
model.add(MaxPooling2D(padding="valid"))
model.add(Conv2D(
    filters=64,
    kernel_size=5
))
# model.add(MaxPooling2D(padding="valid"))
model.add(Flatten())
# model.add(Dense(
#     units=10,
#     activation='relu'
# ))
model.add(Dense(
    units=10,
    activation='softmax'
))
# loss = 
model.compile(optimizer=Adam(lr=.1),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [249]:
# print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[2].get_weights()[0][0,0,0]))
print_gradients = LambdaCallback(on_epoch_end=lambda batch, logs: print(eval(gradients(model.layers[7].output, model.trainable_weights)[0].eval())))
print_last_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[len(model.layers)-1].get_weights()[0][0]))
print_first_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()[0][0]))
print_softmax = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[8].output.eval(session=sess)))


In [231]:
# model.layers[7].get_weights()[0][0]
# # train
# model.layers[7].output
# with sess.as_default():
# gradients(model.layers[7].output, model.trainable_weights)[0].eval()
model.layers[8].output.eval(session=sess)

FailedPreconditionError: Attempting to use uninitialized value dense_38/bias
	 [[node dense_38/bias/read (defined at /Users/mattjones/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:402) ]]

Caused by op 'dense_38/bias/read', defined at:
  File "/Users/mattjones/miniconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/mattjones/miniconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/tornado/platform/asyncio.

In [232]:
y_cat = to_categorical(y, num_classes=10)
# print(y_cat)
model.fit(x,y_cat,batch_size=10, epochs=10, callbacks = [print_softmax])

Epoch 1/10
4/4 [==============================] - 3s 747ms/step - loss: 3.1739 - acc: 0.0000e+00


InvalidArgumentError: You must feed a value for placeholder tensor 'conv2d_74_input' with dtype float and shape [?,128,1288,1]
	 [[node conv2d_74_input (defined at /Users/mattjones/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517) ]]

Caused by op 'conv2d_74_input', defined at:
  File "/Users/mattjones/miniconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/mattjones/miniconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/mattjones/miniconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/Users/mattjones/miniconda3/lib/pytho

In [178]:
# y_cat[0]
predictions = model.predict(x,verbose=1)

2/2 [==============================] - 0s 185ms/step


In [179]:
for i in range(10):
    print(predictions[i])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


IndexError: index 2 is out of bounds for axis 0 with size 2

In [217]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 124, 1284, 16)     416       
_________________________________________________________________
batch_normalization_3 (Batch (None, 124, 1284, 16)     64        
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 62, 642, 16)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 58, 638, 32)       12832     
_________________________________________________________________
batch_normalization_4 (Batch (None, 58, 638, 32)       128       
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 29, 319, 32)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 25, 315, 64)       51264     
__________

In [84]:
model.get_weights()

[array([[[[-7.53649650e-03,  2.39357841e-03, -2.25867890e-02,
           -8.91536623e-02, -2.39291824e-02, -9.55901295e-02,
            1.11219071e-01, -9.41037312e-02, -9.07069817e-02,
            8.37342367e-02,  8.13184157e-02, -9.60851908e-02,
            1.80659741e-02,  7.53376186e-02,  1.24906097e-02,
           -5.33952042e-02],
          [-4.61333431e-02,  2.54439116e-02,  2.44926997e-02,
            4.47189622e-02, -6.92968369e-02,  2.84527950e-02,
           -7.76232257e-02, -6.37239888e-02,  6.07279949e-02,
            4.00772505e-02, -6.96984380e-02, -6.92180544e-02,
           -5.43400645e-02, -8.03170726e-02, -9.74709094e-02,
           -2.56125396e-03]],
 
         [[-6.53964579e-02,  7.81525522e-02, -1.13414280e-01,
            1.09709769e-01,  3.12082879e-02, -5.50335534e-02,
           -4.96483967e-02, -9.94002745e-02, -3.90963741e-02,
            1.86406123e-03, -3.54220010e-02,  3.55952717e-02,
           -7.34537840e-02, -9.67290699e-02, -6.90884069e-02,
         

In [233]:
predictions_test = model.predict(random_test,verbose=1, callbacks=[print_softmax])
for i in range(len(predictions_test)):
    print(predictions_test)

In [253]:
model_fc = Sequential()
model_fc.add(Dense(
    units=100,
    input_shape=(661504,)
))
model_fc.add(Dense(
    units=10,
    activation='softmax'
))
# model_fc.summary()
model_fc.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [254]:
model_fc.fit(raw_y,y_cat,batch_size=1, epochs=3, callbacks=[print_first_weights])

Epoch 1/3
4/4 [==============================] - 9s 2s/step - loss: 8.5786 - acc: 0.5000
[[[-0.00141054  0.03924088  0.00982541 -0.06723718  0.14631334
    0.085247    0.05593383 -0.00029854  0.19803205  0.10322731
    0.19210014  0.12397878  0.05366305 -0.155586   -0.16645752
    0.15057439]]

 [[-0.18966101  0.16679089  0.16780674  0.08289624  0.16095227
   -0.02773692  0.00476591 -0.05361988 -0.08788291  0.11735255
   -0.01341543 -0.13236004  0.06324315 -0.18394592 -0.1097433
   -0.00178529]]

 [[-0.06578009 -0.01824477 -0.05911817  0.19440772 -0.01845913
    0.02624493 -0.16905391 -0.17659797 -0.02613381  0.02919289
    0.12753558 -0.00067317  0.13613641 -0.01125458 -0.11592694
   -0.0182863 ]]

 [[-0.04725617 -0.2088394  -0.09113347 -0.12641612  0.10209908
    0.18108234 -0.213449    0.02864265  0.00988443  0.07360809
    0.01936161 -0.18102069  0.20523158 -0.18567625 -0.01277931
   -0.10194732]]

 [[-0.15339404  0.01524298 -0.18633261 -0.0212782  -0.03636252
   -0.02153432 -0.001